# Example 17 - Three Point Problem and Folded Layers



This example will show how to convert the geological map below using ``GemGIS`` to a `GemPy` model. This example is based on digitized data. The area is 3892 m wide (W-E extent) and 2683 m high (N-S extent). The model represents a coal seam that was encountered at the surface and in boreholes. A second coal seam is located 300 m vertically above the first one. The vertical model extent varies between 0 m and 1000 m.

The map has been georeferenced with QGIS. The stratigraphic boundaries were digitized in QGIS. Strikes lines were digitized in QGIS as well and will be used to calculate orientations for the `GemPy` model. The contour lines were also digitized and will be interpolated with `GemGIS` to create a topography for the model. 

Map Source: An Introduction to Geological Structures and Maps by G.M. Bennison

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread('../data/images/example17/cover_example17.png')
plt.figure(figsize=(10, 10))
imgplot = plt.imshow(img)
plt.axis('off')
plt.tight_layout()

## Licensing

Computational Geosciences and Reservoir Engineering, RWTH Aachen University, Authors: Alexander Juestel. For more information contact: alexander.juestel(at)rwth-aachen.de


This work is licensed under a Creative Commons Attribution 4.0 International License (http://creativecommons.org/licenses/by/4.0/)

## Import GemGIS

If you have installed ``GemGIS`` via pip or conda, you can import ``GemGIS`` like any other package. If you have downloaded the repository, append the path to the directory where the ``GemGIS`` repository is stored and then import ``GemGIS``. 

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import gemgis as gg

## Importing Libraries and loading Data

All remaining packages can be loaded in order to prepare the data and to construct the model. The example data is downloaded from an external server using `pooch`. It will be stored in a data folder in the same directory where this notebook is stored.  

In [ ]:
import geopandas as gpd
import rasterio 

In [ ]:
file_path = '../data/example17_three_point_problem/'

## Creating Digital Elevation Model from Contour Lines

The digital elevation model (DEM) will be created by interpolating contour lines digitized from the georeferenced map using the `SciPy` Radial Basis Function interpolation wrapped in `GemGIS`. The respective function used for that is `gg.vector.interpolate_raster()`. 


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread('../data/images/example17/dem_example17.png')
plt.figure(figsize=(10, 10))
imgplot = plt.imshow(img)
plt.axis('off')
plt.tight_layout()

In [ ]:
topo = gpd.read_file(file_path + 'topo17.shp')
topo.head()

### Interpolating the contour lines

In [ ]:
topo_raster = gg.vector.interpolate_raster(gdf=topo, value='Z', method='rbf', res=10)

### Plotting the raster

In [ ]:
import matplotlib.pyplot as plt

from mpl_toolkits.axes_grid1 import make_axes_locatable

fix, ax = plt.subplots(1, figsize=(10, 10))
topo.plot(ax=ax, aspect='equal', column='Z', cmap='gist_earth')
im = plt.imshow(topo_raster, origin='lower', extent=[0, 3892, 0, 2683], cmap='gist_earth')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar = plt.colorbar(im, cax=cax)
cbar.set_label('Altitude [m]')
ax.set_xlabel('X [m]')
ax.set_ylabel('Y [m]')
ax.set_xlim(0, 3892)
ax.set_ylim(0, 2683)

### Saving the raster to disc

After the interpolation of the contour lines, the raster is saved to disc using `gg.raster.save_as_tiff()`. The function will not be executed as a raster is already provided with the example data. 

### Opening Raster

The previously computed and saved raster can now be opened using rasterio. 

In [ ]:
topo_raster = rasterio.open(file_path + 'raster17.tif')

## Interface Points of stratigraphic boundaries

The interface points will be extracted from LineStrings digitized from the georeferenced map using QGIS. It is important to provide a formation name for each layer boundary. The vertical position of the interface point will be extracted from the digital elevation model using the `GemGIS` function `gg.vector.extract_xyz()`. The resulting GeoDataFrame now contains single points including the information about the respective formation. 


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread('../data/images/example17/interfaces_example17.png')
plt.figure(figsize=(10, 10))
imgplot = plt.imshow(img)
plt.axis('off')
plt.tight_layout()

In [ ]:
interfaces = gpd.read_file(file_path + 'interfaces17.shp')
interfaces.head()

### Extracting Z coordinate from Digital Elevation Model

In [ ]:
interfaces_coords = gg.vector.extract_xyz(gdf=interfaces, dem=topo_raster)
interfaces_coords = interfaces_coords.sort_values(by='formation', ascending=True)
interfaces_coords.head()

In [ ]:
points = gpd.read_file(file_path + 'points17.shp')
points

In [ ]:
points_coords = gg.vector.extract_xy(gdf=points)
points_coords

In [ ]:
import pandas as pd

interfaces_coords = pd.concat([interfaces_coords, points_coords])
interfaces_coords = interfaces_coords[interfaces_coords['formation'].isin(['Coal', 'Nodules'])].reset_index()
interfaces_coords.head()

### Creating data for the coal seam 300 m heigher

In [ ]:
interfaces_coal300 = interfaces_coords[interfaces_coords['formation']=='Coal']
interfaces_coal300['Z'] = interfaces_coal300['Z']+300
interfaces_coal300['formation'] = 'Coal300'
interfaces_coal300

### Merging the interface data

In [ ]:
interfaces_coords = pd.concat([interfaces_coal300, interfaces_coords])
interfaces_coords = interfaces_coords[interfaces_coords['formation'].isin(['Coal300', 'Coal', 'Nodules'])].reset_index()
interfaces_coords.head()

### Plotting the Interface Points

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 10))

interfaces.plot(ax=ax, column='formation', legend=True, aspect='equal')
interfaces_coords.plot(ax=ax, column='formation', legend=True, aspect='equal')
points_coords.plot(ax=ax, column='formation', legend=False, aspect='equal')
plt.grid()
ax.set_xlabel('X [m]')
ax.set_ylabel('Y [m]')
ax.set_xlim(0, 3892)
ax.set_ylim(0, 2683)

## Orientations from Strike Lines

Strike lines connect outcropping stratigraphic boundaries (interfaces) of the same altitude. In other words: the intersections between topographic contours and stratigraphic boundaries at the surface. The height difference and the horizontal difference between two digitized lines is used to calculate the dip and azimuth and hence an orientation that is necessary for `GemPy`. In order to calculate the orientations, each set of strikes lines/LineStrings for one formation must be given an id number next to the altitude of the strike line. The id field is already predefined in QGIS. The strike line with the lowest altitude gets the id number `1`, the strike line with the highest altitude the the number according to the number of digitized strike lines. It is currently recommended to use one set of strike lines for each structural element of one formation as illustrated. 


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread('../data/images/example17/orientations_example17.png')
plt.figure(figsize=(10, 10))
imgplot = plt.imshow(img)
plt.axis('off')
plt.tight_layout()

In [ ]:
strikes = gpd.read_file(file_path + 'strikes17.shp')
strikes.head()

### Calculate Orientations for each formation

In [ ]:
orientations_coal1 = gg.vector.calculate_orientations_from_strike_lines(gdf=strikes[strikes['formation'] == 'Coal1'].sort_values(by='Z', ascending=True).reset_index())
orientations_coal1

In [ ]:
orientations_coal2 = gg.vector.calculate_orientations_from_strike_lines(gdf=strikes[strikes['formation'] == 'Coal2'].sort_values(by='Z', ascending=True).reset_index())
orientations_coal2

In [ ]:
orientations_nodules = gg.vector.calculate_orientations_from_strike_lines(gdf=strikes[strikes['formation'] == 'Nodules'].sort_values(by='Z', ascending=True).reset_index())
orientations_nodules

In [ ]:
orientations_coal3 = gg.vector.calculate_orientation_for_three_point_problem(gdf=points)
orientations_coal3['Z'] = orientations_coal3['Z'].astype(float)
orientations_coal3['azimuth'] = orientations_coal3['azimuth'].astype(float)
orientations_coal3['dip'] = orientations_coal3['dip'].astype(float)
orientations_coal3['dip'] = 180 - orientations_coal3['dip']
orientations_coal3['azimuth'] = 180 - orientations_coal3['azimuth']
orientations_coal3['polarity'] = orientations_coal3['polarity'].astype(float)
orientations_coal3['X'] = orientations_coal3['X'].astype(float)
orientations_coal3['Y'] = orientations_coal3['Y'].astype(float)
orientations_coal3

### Merging Orientations

In [ ]:
import pandas as pd
orientations = pd.concat([orientations_coal1, orientations_coal2, orientations_coal3, orientations_nodules]).reset_index()
orientations['formation'] = ['Coal', 'Coal', 'Coal', 'Nodules']
orientations = orientations[orientations['formation'].isin(['Coal', 'Nodules'])]
orientations

### Plotting the Orientations

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 10))

interfaces.plot(ax=ax, column='formation', legend=True, aspect='equal')
interfaces_coords.plot(ax=ax, column='formation', legend=True, aspect='equal')
orientations.plot(ax=ax, color='red', aspect='equal')
plt.grid()
ax.set_xlabel('X [m]')
ax.set_ylabel('Y [m]')
ax.set_xlim(0, 3892)
ax.set_ylim(0, 2683)

## GemPy Model Construction

The structural geological model will be constructed using the `GemPy` package. 

In [ ]:
import gempy as gp

### Creating new Model

In [ ]:
geo_model = gp.create_model('Model17')
geo_model

### Initiate Data

In [ ]:
gp.init_data(geo_model, [0, 3892, 0, 2683, 0, 1000], [100, 100, 100],
             surface_points_df=interfaces_coords[interfaces_coords['Z'] != 0],
             orientations_df=orientations,
             default_values=True)

### Model Surfaces

In [ ]:
geo_model.surfaces

### Mapping the Stack to Surfaces

In [ ]:
gp.map_stack_to_surfaces(geo_model,
                         {
                          'Strata1': ('Coal300', 'Coal', 'Nodules'),   
                         },
                         remove_unused_series=True)
geo_model.add_surfaces('Basement')

### Showing the Number of Data Points

In [ ]:
gg.utils.show_number_of_data_points(geo_model=geo_model)

### Loading Digital Elevation Model

In [ ]:
geo_model.set_topography(
    source='gdal', filepath=file_path + 'raster17.tif')

### Defining Custom Section

In [ ]:
custom_section = gpd.read_file(file_path + 'customsection17.shp')
custom_section_dict = gg.utils.to_section_dict(custom_section, section_column='name')
geo_model.set_section_grid(custom_section_dict)

In [ ]:
gp.plot.plot_section_traces(geo_model)

### Plotting Input Data

In [ ]:
gp.plot_2d(geo_model, direction='z', show_lith=False, show_boundaries=False)
plt.grid()

In [ ]:
gp.plot_3d(geo_model, image=False, plotter_type='basic', notebook=True)

### Setting the Interpolator

In [ ]:
gp.set_interpolator(geo_model,
                    compile_theano=True,
                    theano_optimizer='fast_compile',
                    verbose=[],
                    update_kriging=False
                    )

### Computing Model

In [ ]:
sol = gp.compute_model(geo_model, compute_mesh=True)

### Plotting Cross Sections

In [ ]:
gp.plot_2d(geo_model, section_names=['Section1'], show_topography=True, show_data=False)

In [ ]:
gp.plot_2d(geo_model, direction=['x', 'x', 'y', 'y'], cell_number=[25, 75, 25, 75], show_topography=True, show_data=False)

### Plotting 3D Model

In [ ]:
gpv = gp.plot_3d(geo_model, image=False, show_topography=True,
                 plotter_type='basic', notebook=True, show_lith=True)